In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib.pyplot as plt
import seaborn as sns
import folium


In [ ]:
# მონაცემების ჩატვირთვა
train_df =  pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv', nrows = 10_000)
test_df =  pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')


In [ ]:
test_df.shape


In [ ]:
test_df.describe()


In [ ]:
train_df.head()


In [ ]:
train_df.shape


In [ ]:
train_df.dtypes


In [ ]:
train_df.describe()


In [ ]:
print(train_df.isnull().sum())


In [ ]:
# გადავყაროთ missing values, რადგან დატასეტის მცირე ნაწილს შეადგენენ
print('Old size: %d' % len(train_df))
train_df = train_df.dropna(how = 'any', axis = 'rows')
print('New size: %d' % len(train_df))


In [ ]:
print(len(train_df[train_df.fare_amount > 100]))


In [ ]:
# გადავყაროთ არადადებითი თანხები და 100 მეტი თანხები
print('Old size: %d' % len(train_df))
train_df = train_df[(train_df.fare_amount>0) & (train_df.fare_amount<=100)]
print('New size: %d' % len(train_df))


In [ ]:
train_df.describe()


In [ ]:
print(len(train_df[train_df.passenger_count > 6]))


In [ ]:
print(len(train_df[train_df.passenger_count == 0]))


In [ ]:
# გადავყაროთ 6-ზე მეტ მგზავრიანი მონაცემები

print('Old size: %d' % len(train_df))
old_len = len(train_df)
train_df = train_df[(train_df.passenger_count<=6) & (train_df.passenger_count>0)]
print(f'New size: {len(train_df)}, removed: {old_len - len(train_df)}')


In [ ]:
train_df.describe()


In [ ]:
plt.hist(train_df['fare_amount'], bins=10, color='skyblue', edgecolor='black')
plt.title('Distribution of Fare Amounts')
plt.xlabel('Fare Amount')
plt.ylabel('Frequency')
plt.show()


In [ ]:
train_df.describe()


In [ ]:
# ტესტსეტის მიხედვით ვიპოვოთ ნიუ-იორკის მიახლოებითი კოორდინატები
# min_longitude = min(test_df['pickup_longitude'].min(), test_df['dropoff_longitude'].min())
# max_longitude = max(test_df['pickup_longitude'].max(), test_df['dropoff_longitude'].max())

# min_latitude = min(test_df['pickup_latitude'].min(), test_df['dropoff_latitude'].min())
# max_latitude = max(test_df['pickup_latitude'].max(), test_df['dropoff_latitude'].max())

# ნიუ იორკის მიახლოებითი კოორდინატები
min_longitude = -74.5
max_longitude = -72.8
min_latitude = 40.5
max_latitude = 41.8

#-74.5, -72.8, 40.5, 41.8

print("Minimum Longitude:", min_longitude)
print("Maximum Longitude:", max_longitude)
print("Minimum Latitude:", min_latitude)
print("Maximum Latitude:", max_latitude)


In [ ]:
train_df = train_df[
    (train_df['pickup_longitude'] >= min_longitude) &
    (train_df['pickup_longitude'] <= max_longitude) &
    (train_df['pickup_latitude'] >= min_latitude) &
    (train_df['pickup_latitude'] <= max_latitude)
]

train_df = train_df[
    (train_df['dropoff_longitude'] >= min_longitude) &
    (train_df['dropoff_longitude'] <= max_longitude) &
    (train_df['dropoff_latitude'] >= min_latitude) &
    (train_df['dropoff_latitude'] <= max_latitude)
]

train_df.describe()


In [ ]:
from geopy.distance import geodesic

from math import radians, sin, cos, sqrt, atan2

# დავითვალოთ მანძილები და დავამატოთ ახალი ფიჩერის სახით

def haversine_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  
    
    return distance

distances = train_df.apply(lambda row: haversine_distance(
                                row['pickup_latitude'], row['pickup_longitude'],
                                row['dropoff_latitude'], row['dropoff_longitude']), axis=1)

train_df['distance'] = distances

plt.hist(list(filter(lambda x: x <= 30, distances)), bins=100, color='skyblue', edgecolor='black')
plt.title('Distribution of Distances between Pickup and Dropoff Locations')
plt.xlabel('Distance (km)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
train_df.distance.describe()


In [ ]:
train_df['fare_per_km'] = train_df.distance/train_df.fare_amount
train_df.fare_per_km.describe()


In [ ]:
train_df = train_df[(train_df['fare_per_km'] <= 0.5) & (train_df['fare_per_km'] > 0)]
plt.hist(train_df['fare_per_km'], bins=10, color='skyblue', edgecolor='black')
plt.title('Fare per km distribuiton')
plt.xlabel('Fare per km')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# დავამატოთ წელი, კვირის დღე და საათი ფიჩერებად
pickup_datetime = pd.to_datetime(train_df['pickup_datetime'])
train_df['year'] = pickup_datetime.apply(lambda t: t.year)
train_df['weekday'] = pickup_datetime.apply(lambda t: t.weekday())
train_df['hour'] = pickup_datetime.apply(lambda t: t.hour)


In [ ]:
plt.hist(train_df['weekday'], bins=10, color='skyblue', edgecolor='black')
plt.title('Weekday distribution')
plt.xlabel('Weekday')
plt.ylabel('Frequency')
plt.show()


In [ ]:
plt.hist(train_df['hour'], bins=20, color='skyblue', edgecolor='black')
plt.title('Hour distribution')
plt.xlabel('Hour')
plt.ylabel('Frequency')
plt.show()


In [ ]:
hourly_fare_mean = train_df.groupby('hour')['fare_per_km'].mean()
print(hourly_fare_mean)


In [ ]:
import matplotlib.pyplot as plt

# Example Series hourly_fare_mean containing mean fare amount for each hour
# Replace hourly_fare_mean with your actual Series

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(hourly_fare_mean.index, hourly_fare_mean.values, marker='o', linestyle='-')
plt.title('Mean Fare per km by Hour')
plt.xlabel('Hour')
plt.ylabel('Mean Fare Amount')
plt.xticks(hourly_fare_mean.index)  # Ensure ticks are at each hour
plt.grid(True)
plt.show()


In [ ]:
weekday_fare_mean = train_df.groupby('weekday')['fare_per_km'].mean()
import matplotlib.pyplot as plt

# Example Series hourly_fare_mean containing mean fare amount for each hour
# Replace hourly_fare_mean with your actual Series

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(weekday_fare_mean.index, weekday_fare_mean.values, marker='o', linestyle='-')
plt.title('Mean Fare per km by weekday')
plt.xlabel('Weekday')
plt.ylabel('Mean Fare Amount')
plt.xticks(weekday_fare_mean.index)  # Ensure ticks are at each hour
plt.grid(True)
plt.show()


In [ ]:
yearly_fare_mean = train_df.groupby('year')['fare_per_km'].mean()
import matplotlib.pyplot as plt

# Example Series hourly_fare_mean containing mean fare amount for each hour
# Replace hourly_fare_mean with your actual Series

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(yearly_fare_mean.index, yearly_fare_mean.values, marker='o', linestyle='-')
plt.title('Mean Fare per km by year')
plt.xlabel('year')
plt.ylabel('Mean Fare Amount')
plt.xticks(yearly_fare_mean.index)  # Ensure ticks are at each hour
plt.grid(True)
plt.show()


In [ ]:
train_df['high_fare_days'] = train_df['weekday'].isin([0, 6, 5]).astype(int)
train_df['low_fare_days'] = train_df['weekday'].isin([1, 2, 3, 4]).astype(int)

train_df['low_fare_hours'] = ((train_df['hour'] >= 8) & (train_df['hour'] <= 19)).astype(int)
train_df['high_fare_hours'] = ((train_df['hour'] < 8) | (train_df['hour'] > 19)).astype(int)

train_df['high_fare_years'] = (train_df['year'] <= 2012).astype(int)
train_df['low_fare_years'] = (train_df['year'] > 2012).astype(int)

train_df['monday'] = (train_df['weekday'] == 0).astype(int)
train_df['tuesday'] = (train_df['weekday'] == 1).astype(int)
train_df['wednesday'] = (train_df['weekday'] == 2).astype(int)
train_df['thursday'] = (train_df['weekday'] == 3).astype(int)
train_df['friday'] = (train_df['weekday'] == 4).astype(int)
train_df['saturday'] = (train_df['weekday'] == 5).astype(int)
train_df['sunday'] = (train_df['weekday'] == 6).astype(int)
train_df.describe()
print(train_df['sunday'].sum())
print(train_df['monday'].sum())
print(train_df['friday'].sum())
print(train_df['tuesday'].sum())
print(train_df['wednesday'].sum())
print(train_df['saturday'].sum())


In [ ]:
distances = test_df.apply(lambda row: haversine_distance(
                                row['pickup_latitude'], row['pickup_longitude'],
                                row['dropoff_latitude'], row['dropoff_longitude']), axis=1)

test_df['distance'] = distances
pickup_datetime = pd.to_datetime(test_df['pickup_datetime'])
print(pickup_datetime)
test_df['year'] = pickup_datetime.apply(lambda t: t.year)
test_df['weekday'] = pickup_datetime.apply(lambda t: t.weekday())
test_df['hour'] = pickup_datetime.apply(lambda t: t.hour)

test_df['high_fare_days'] = test_df['weekday'].isin([0, 6, 5]).astype(int)
test_df['low_fare_days'] = test_df['weekday'].isin([1, 2, 3, 4]).astype(int)

test_df['low_fare_hours'] = ((test_df['hour'] >= 8) & (test_df['hour'] <= 19)).astype(int)
test_df['high_fare_hours'] = ((test_df['hour'] < 8) | (test_df['hour'] > 19)).astype(int)

test_df['high_fare_years'] = (test_df['year'] <= 2012).astype(int)
test_df['low_fare_years'] = (test_df['year'] > 2012).astype(int)

test_df['monday'] = (test_df['weekday'] == 0).astype(int)
test_df['tuesday'] = (test_df['weekday'] == 1).astype(int)
test_df['wednesday'] = (test_df['weekday'] == 2).astype(int)
test_df['thursday'] = (test_df['weekday'] == 3).astype(int)
test_df['friday'] = (test_df['weekday'] == 4).astype(int)
test_df['saturday'] = (test_df['weekday'] == 5).astype(int)
test_df['sunday'] = (test_df['weekday'] == 6).astype(int)
test_df.describe()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = train_df[[
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "passenger_count",
    "distance",
    "year",
#    "weekday",
    "hour",
    "monday",
    "tuesday",
    "wednesday",
    "thursday",
    "friday",
    "saturday",
    "sunday",
    "high_fare_days",
    "low_fare_days",
    "high_fare_years",
    "low_fare_years",
    "high_fare_hours",
    "low_fare_hours",
]]

y = train_df["fare_amount"]

X_test = test_df.drop(columns=["key", "pickup_datetime"])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42, shuffle=True)

# Instantiate the linear regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the validation data
y_pred_valid = model.predict(X_valid)

# Evaluate model performance on validation data
rmse_valid = mean_squared_error(y_valid, y_pred_valid, squared=False)
print("Mean Squared Error on Validation Data:", rmse_valid)

# Make predictions on the test data
#y_pred_test = model.predict(X_test)

# # Evaluate model performance on test data (optional)
#rmse_test = mean_squared_error(y_test, y_pred_test)
#print("Mean Squared Error on Test Data:", rmse_test)


In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error

# X = train_df.drop(columns=["key", "fare_amount", "pickup_datetime", "fare_per_km"])

# y = train_df["fare_amount"]

# X_test = test_df.drop(columns=["key", "pickup_datetime"])
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42, shuffle=True)

# # Instantiate the linear regression model
# model = LinearRegression()

# # Train the model on the training data
# model.fit(X_train, y_train)

# # Make predictions on the validation data
# y_pred_valid = model.predict(X_valid)

# # Evaluate model performance on validation data
# rmse_valid = mean_squared_error(y_valid, y_pred_valid, squared = False)
# print("Mean Squared Error on Validation Data:", rmse_valid)


In [ ]:
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# X = train_df[[
#     "pickup_longitude",
#     "pickup_latitude",
#     "dropoff_longitude",
#     "dropoff_latitude",
#     "passenger_count",
#     "distance",
#     "year",
#     "weekday",
#     "hour",
#     "monday",
#     "tuesday",
#     "wednesday",
#     "thursday",
#     "friday",
#     "saturday",
#     "sunday",
#     "high_fare_days",
#     "high_fare_years",
#     "low_fare_hours",
# ]]

# y = train_df["fare_amount"]

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, shuffle=True)

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dvalid = xgb.DMatrix(X_valid, label=y_valid)

# params = {
#     "objective": "reg:squarederror",  
#     "eval_metric": "rmse",             
#     "random_state": 42,
#     'max_depth': 5,
#     "eta": 0.1
# }

# num_rounds = 500 
# model = xgb.train(params, dtrain, num_rounds, evals=[(dvalid, "Validation")], early_stopping_rounds=10, verbose_eval=True)

# y_pred = model.predict(dvalid)

# rmse = mean_squared_error(y_valid, y_pred, squared = False)
# print("Mean Squared Error on Validation Set:", rmse)


In [ ]:
# import xgboost as xgb        
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# X = train_df[[
#     "pickup_longitude",
#     "pickup_latitude",
#     "dropoff_longitude",
#     "dropoff_latitude",
#     "passenger_count",
# #     "distance",
# #     "year",
# #     "weekday",
# #     "hour",
# #     "monday",
# #     "tuesday",
# #     "wednesday",
# #     "thursday",
# #     "friday",
# #     "saturday",
# #     "sunday",
# #     "high_fare_days",
# #     "high_fare_years",
# #     "low_fare_hours",
# ]]

# y = train_df["fare_amount"]

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, shuffle=True)

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dvalid = xgb.DMatrix(X_valid, label=y_valid)

# params = {
#     "objective": "reg:squarederror",  
#     "eval_metric": "rmse",             
#     "random_state": 42,
#     'max_depth': 6,
#     "eta": 0.1
# }

# n_estimators = 500 
# model = xgb.train(params, dtrain, n_estimators, evals=[(dvalid, "Validation")], early_stopping_rounds=5, verbose_eval=True)

# y_pred = model.predict(dvalid)

# rmse = mean_squared_error(y_valid, y_pred, squared = False)
# print("Mean Squared Error on Validation Set:", rmse)


In [ ]:
# import xgboost as xgb        
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# X = train_df[[
#     "pickup_longitude",
#     "pickup_latitude",
#     "dropoff_longitude",
#     "dropoff_latitude",
#     "passenger_count",
#     "distance",
#     "year",
#     "weekday",
#     "hour",
# #     "monday",
# #     "tuesday",
# #     "wednesday",
# #     "thursday",
# #     "friday",
# #     "saturday",
# #     "sunday",
# #     "high_fare_days",
# #     "high_fare_years",
# #     "low_fare_hours",
# ]]

# y = train_df["fare_amount"]

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, shuffle=True)

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dvalid = xgb.DMatrix(X_valid, label=y_valid)

# params = {
#     "objective": "reg:squarederror",  
#     "eval_metric": "rmse",             
#     "random_state": 42,
#     'max_depth': 6,
#     "eta": 0.1
# }

# n_estimators = 500 
# model = xgb.train(params, dtrain, n_estimators, evals=[(dvalid, "Validation")], early_stopping_rounds=5, verbose_eval=True)

# y_pred = model.predict(dvalid)

# rmse = mean_squared_error(y_valid, y_pred, squared = False)
# print("Mean Squared Error on Validation Set:", rmse)


In [ ]:
# import xgboost as xgb        
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# X = train_df[[
#     "pickup_longitude",
#     "pickup_latitude",
#     "dropoff_longitude",
#     "dropoff_latitude",
#     "passenger_count",
#     "distance",
#     "year",
# #    "weekday",
#     "hour",
#     "monday",
#     "tuesday",
#     "wednesday",
#     "thursday",
#     "friday",
#     "saturday",
#     "sunday",
# #     "high_fare_days",
# #     "high_fare_years",
# #     "low_fare_hours",
# ]]

# y = train_df["fare_amount"]

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, shuffle=True)

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dvalid = xgb.DMatrix(X_valid, label=y_valid)

# params = {
#     "objective": "reg:squarederror",  
#     "eval_metric": "rmse",             
#     "random_state": 42,
#     'max_depth': 6,
#     "eta": 0.1
# }

# n_estimators = 500 
# model = xgb.train(params, dtrain, n_estimators, evals=[(dvalid, "Validation")], early_stopping_rounds=5, verbose_eval=True)

# y_pred = model.predict(dvalid)

# rmse = mean_squared_error(y_valid, y_pred, squared = False)
# print("Mean Squared Error on Validation Set:", rmse)


In [ ]:
# import xgboost as xgb        
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# X = train_df[[
#     "pickup_longitude",
#     "pickup_latitude",
#     "dropoff_longitude",
#     "dropoff_latitude",
#     "passenger_count",
#     "distance",
#     "year",
# #    "weekday",
#     "hour",
#     "monday",
#     "tuesday",
#     "wednesday",
#     "thursday",
#     "friday",
#     "saturday",
#     "sunday",
# #     "high_fare_days",
# #     "high_fare_years",
# #     "low_fare_hours",
# ]]

# y = train_df["fare_amount"]

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, shuffle=True)

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dvalid = xgb.DMatrix(X_valid, label=y_valid)

# params = {
#     "objective": "reg:squarederror",  
#     "eval_metric": "rmse",             
#     "random_state": 42,
#     'max_depth': 6,
#     "eta": 0.01
# }

# n_estimators = 1000 
# model = xgb.train(params, dtrain, n_estimators, evals=[(dvalid, "Validation")], early_stopping_rounds=5, verbose_eval=True)

# y_pred = model.predict(dvalid)

# rmse = mean_squared_error(y_valid, y_pred, squared = False)
# print("Mean Squared Error on Validation Set:", rmse)


In [ ]:
# import xgboost as xgb        
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# X = train_df[[
#     "pickup_longitude",
#     "pickup_latitude",
#     "dropoff_longitude",
#     "dropoff_latitude",
#     "passenger_count",
#     "distance",
#     "year",
# #    "weekday",
#     "hour",
#     "monday",
#     "tuesday",
#     "wednesday",
#     "thursday",
#     "friday",
#     "saturday",
#     "sunday",
#     "high_fare_days",
#     "low_fare_days",
#     "high_fare_years",
#     "low_fare_years",
#     "high_fare_hours",
#     "low_fare_hours",
# ]]

# y = train_df["fare_amount"]

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, shuffle=True)

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dvalid = xgb.DMatrix(X_valid, label=y_valid)

# params = {
#     "objective": "reg:squarederror",  
#     "eval_metric": "rmse",             
#     "random_state": 42,
#     'max_depth': 6,
#     "eta": 0.1
# }

# n_estimators = 500 
# model = xgb.train(params, dtrain, n_estimators, evals=[(dvalid, "Validation")], early_stopping_rounds=5, verbose_eval=True)

# y_pred = model.predict(dvalid)

# rmse = mean_squared_error(y_valid, y_pred, squared = False)
# print("Mean Squared Error on Validation Set:", rmse)


In [ ]:
X_test = test_df[[
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "passenger_count",
    "distance",
    "year",
#    "weekday",
    "hour",
    "monday",
    "tuesday",
    "wednesday",
    "thursday",
    "friday",
    "saturday",
    "sunday",
    "high_fare_days",
    "low_fare_days",
    "high_fare_years",
    "low_fare_years",
    "high_fare_hours",
    "low_fare_hours",
]]

#dtest = xgb.DMatrix(X_test)
# Make predictions on the test data
y_pred_test = model.predict(X_test)
submission = pd.DataFrame(
    {'key': test_df.key, 'fare_amount': y_pred_test},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission.csv', index = False)

print(os.listdir('.'))
